In [1]:
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')

In [8]:
x = []
y = []

face_x = []
face_y = []
face_w = []
face_h = []

left_x = []
left_y = []
left_w = []
left_h = []

right_x = []
right_y = []
right_w = []
right_h = []

left_iris_x = []
left_iris_y = []
right_iris_x = []
right_iris_y = []

In [9]:
source = "dataset"
Kernal = np.ones((3, 3), np.uint8)

for filename in os.listdir(source):
    features = []
    file = filename.split('.')
    features.append(int(file[0]))
    features.append(int(file[1]))
    img = cv2.imread(os.path.join(source,filename))
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    for (fx,fy,fw,fh) in faces:
        features.extend([fx, fy, fw, fh])
        roi_gray = gray_frame[fy:fy+fh, fx:fx+fw]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5) # detect eyes
        width = np.size(roi_gray, 1) # get face frame width
        height = np.size(roi_gray, 0) # get face frame height
        left_eye_status = False
        right_eye_status = False
        for (ex, ey, ew, eh) in eyes:
            if ey > height / 2:
                pass
            eyecenter = ex + ew / 2  # get the eye center
            if eyecenter < width * 0.5:
                left_eye = roi_gray[ey:ey + eh, ex:ex + ew]
                left_eye = cv2.resize(left_eye, dsize=(100, 100))
                left_eye_status = [ex, ey, ew, eh]
                ret, binary = cv2.threshold(left_eye, 60, 255, cv2.THRESH_BINARY_INV)
                width, height = binary.shape
                binary = binary[int(0.4 * height):height, :]
                opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, Kernal)  ##Opening Morphology
                dilate = cv2.morphologyEx(opening, cv2.MORPH_DILATE, Kernal)  ##Dilate Morphology
                contours, hierarchy = cv2.findContours(dilate, cv2.RETR_TREE,  ##Find contours
                                                        cv2.CHAIN_APPROX_NONE)
                if len(contours) != 0:
                    cnt = contours[0]
                    M1 = cv2.moments(cnt)
                    Cx1 = int(M1['m10'] / M1['m00'])        ##Find center of the contour
                    Cy1 = int(M1['m01'] / M1['m00'])
                    croppedImagePixelLength = int(0.4*height)       ##Number of pixels we cropped from the image
                    center1 = (int(Cx1+fx+ex), int(Cy1+fy+ey+croppedImagePixelLength))    ##Center coordinates
                    left_eye_status.append(center1[0])
                    left_eye_status.append(center1[1])
            if eyecenter > width * 0.5:
                right_eye = roi_gray[ey:ey + eh, ex:ex + ew]
                right_eye = cv2.resize(right_eye, dsize=(100, 100))
                right_eye_status = [ex, ey, ew, eh]
                ret, binary = cv2.threshold(right_eye, 60, 255, cv2.THRESH_BINARY_INV)
                width, height = binary.shape
                binary = binary[int(0.4 * height):height, :]
                opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, Kernal)  ##Opening Morphology
                dilate = cv2.morphologyEx(opening, cv2.MORPH_DILATE, Kernal)  ##Dilate Morphology
                contours, hierarchy = cv2.findContours(dilate, cv2.RETR_TREE,  ##Find contours
                                                        cv2.CHAIN_APPROX_NONE)
                if len(contours) != 0:
                    cnt = contours[0]
                    M1 = cv2.moments(cnt)
                    Cx1 = int(M1['m10'] / M1['m00'])        ##Find center of the contour
                    Cy1 = int(M1['m01'] / M1['m00'])
                    croppedImagePixelLength = int(0.4*height)       ##Number of pixels we cropped from the image
                    center1 = (int(Cx1+fx+ex), int(Cy1+fy+ey+croppedImagePixelLength))    ##Center coordinates
                    right_eye_status.append(center1[0])
                    right_eye_status.append(center1[1])
        if left_eye_status and right_eye_status:
            features.extend(left_eye_status[:-2])
            features.extend(right_eye_status[:-2])
            features.extend(left_eye_status[-2:])
            features.extend(right_eye_status[-2:])
    if len(features) == 18:
        x.append(features[0])
        y.append(features[1])
        face_x.append(features[2])
        face_y.append(features[3])
        face_w.append(features[4])
        face_h.append(features[5])
        left_x.append(features[6])
        left_y.append(features[7])
        left_w.append(features[8])
        left_h.append(features[9])
        right_x.append(features[10])
        right_y.append(features[11])
        right_w.append(features[12])
        right_h.append(features[13])
        left_iris_x.append(features[14])
        left_iris_y.append(features[15])        
        right_iris_x.append(features[16])
        right_iris_y.append(features[17])            

In [10]:
df = pd.DataFrame({
    'x': x,
    'y': y,
    'face_x': face_x,
    'face_y': face_y,
    'face_w': face_w,
    'face_h': face_h,
    'left_x': left_x,
    'left_y': left_y,
    'left_w': left_w,
    'left_h': left_h,
    'right_x': right_x,
    'right_y': right_y,
    'right_w': right_w,
    'right_h': right_h,
    'left_iris_x': left_iris_x,
    'left_iris_y': left_iris_y,
    'right_iris_x': right_iris_x,
    'right_iris_y': right_iris_y
})

In [11]:
df

,x,y,face_x,face_y,face_w,face_h,left_x,left_y,left_w,left_h,right_x,right_y,right_w,right_h,left_iris_x,left_iris_y,right_iris_x,right_iris_y
0,0,0,271,196,216,216,37,47,50,50,126,59,44,44,398,304,408,316
1,0,0,317,196,228,228,33,55,56,56,86,111,41,41,412,302,464,360
2,0,1,316,190,217,217,40,47,49,49,101,59,60,60,447,302,500,295
3,0,101,285,124,168,168,26,50,40,40,94,47,41,41,395,215,386,225
4,0,120,249,258,181,181,28,42,46,46,70,44,60,60,350,353,413,347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13439,998,242,234,169,172,172,28,48,45,45,94,44,41,41,327,269,373,276
13440,998,343,233,209,164,164,25,49,38,38,88,39,41,41,301,310,339,308
13441,998,355,259,151,235,235,55,64,53,53,129,64,60,60,359,276,440,289
13442,999,1,219,175,221,221,44,51,50,50,135,65,49,49,334,279,436,300


In [12]:
df.to_excel('regression_data.xlsx')
df.to_csv('regression_data.csv')

In [13]:
from sklearn.model_selection import train_test_split

x = df[list(df.columns)[2:]]
y = df[['x', 'y']]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
X_train.shape

(10755, 16)

In [15]:
y_train.shape

(10755, 2)

In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [17]:
y_test_pred = model.predict(X_test)

In [19]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

print("Test Set: ")
print(f"R2 Score : ", r2_score(y_test, y_test_pred))
print(f"Mean Squared Error : ", mean_squared_error(y_test, y_test_pred))
print(f"Root Mean Squared Error : ", np.sqrt(mean_squared_error(y_test, y_test_pred)))
print(f"Mean Absolute Error : ", mean_absolute_error(y_test, y_test_pred))

Test Set: 
R2 Score :  0.22925612248908428
Mean Squared Error :  78485.97556908189
Root Mean Squared Error :  280.15348573430583
Mean Absolute Error :  228.42108516665667


In [31]:
comp = pd.concat([y_test, y_test_pred])

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [33]:
type(y_test_pred)

numpy.ndarray

In [34]:
y_test_pred

array([[2033.68663862, 1101.71443224],
       [ 825.39678421,  431.51077323],
       [ 767.3931876 ,  326.12613672],
       ...,
       [ 763.62223201,  421.84960733],
       [ 734.93978573,  343.29806756],
       [ 657.81720433,  307.20108783]])

In [52]:
x_pred = []
y_pred = []
for row in y_test_pred:
    x_pred.append(row[0])
    y_pred.append(row[1])
    
y_test_pred_df = pd.DataFrame({
    'x_pred': x_pred,
    'y_pred': y_pred
})
y_test_pred_df

,x_pred,y_pred
0,2033.686639,1101.714432
1,825.396784,431.510773
2,767.393188,326.126137
3,463.460840,193.955800
4,728.469382,367.312836
...,...,...
2684,761.449318,251.664665
2685,904.514417,438.309339
2686,763.622232,421.849607
2687,734.939786,343.298068


In [54]:
y_test_pred_df.insert(loc=2, column='x_actual', value=pd.Series(y_test['x'], index=y_test_pred_df.index))
y_test_pred_df.insert(loc=3, column='y_actual', value=pd.Series(y_test['y'], index=y_test_pred_df.index))
y_test_pred_df

,x_pred,y_pred,x_actual,y_actual
0,2033.686639,1101.714432,0.0,0.0
1,825.396784,431.510773,NaN,NaN
2,767.393188,326.126137,NaN,NaN
3,463.460840,193.955800,NaN,NaN
4,728.469382,367.312836,NaN,NaN
...,...,...,...,...
2684,761.449318,251.664665,NaN,NaN
2685,904.514417,438.309339,NaN,NaN
2686,763.622232,421.849607,NaN,NaN
2687,734.939786,343.298068,NaN,NaN


In [57]:
y_test_pred_df = y_test_pred_df.dropna()
y_test_pred_df

,x_pred,y_pred,x_actual,y_actual
0,2033.686639,1101.714432,0.0,0.0
8,808.274660,333.491244,0.0,140.0
14,580.320078,291.412858,0.0,161.0
15,610.788244,274.474054,0.0,180.0
17,976.083092,405.113146,0.0,181.0
...,...,...,...,...
2673,572.028487,359.844713,1221.0,470.0
2676,375.414425,515.187936,1223.0,238.0
2677,549.406139,361.327809,1223.0,45.0
2680,792.014216,487.141599,1224.0,637.0
